# SCM.256 Final Project - Milestone 2

### Jesus Madrid - MIT ID 928576183
### Andrew Min    - MIT ID 922931468

## Project Objective:
To build a linear regression model in order to predict the total amount of a sale given a set of independent variables from the Massachisetts transactional and the American Comunnity Census 2017 datasets. 

## The model: 
the model will be created from the following variables:  
![regression_model](img/regression_model.png)

y is the dependet variable and all Xi are independent variables associated with the customer, the store and zip code information. 

## Project Schema
The following schema shows the relation between all databases. from the American Census Dataset we only want the columns related with the Poverty Rate and % of the population over 25 years old per Zip Code
![project schema](https://drive.google.com/open?id=1i2GljQ8si8cKKiYJhHMTyUfQpwbBK3Zz)

## Milestone 2 Objectives:

> #### 1. Clean the data: remove duplicates, fill NaN values, fix datatypes
> #### 2. Merge all tables together in a single sales DataFrame following the schema
> #### 3. Group all sales by Sale_ID
> #### 4. Create X and y DataFrames for the ML model
> #### 5. Apply one-hot encoding to categorical variables
> #### 6. Normalize numerical independent variables

After these procedures the data will be ready to apply a linear regression Machine Learning Algorithm

In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Clean the data: remove duplicates, fill NaN values, fix datatypes

## 1.1 Massachusetts Transactional Dataset

### 1.1.1 Reading the Data

In [2]:
# Read in the Massachusetts transactional data indivually by worksheet within the excel workbook
xlsx = pd.ExcelFile('Massachusetts Dataset/Massachusetts_Transaction_Database.xlsx')
Customers = pd.read_excel(xlsx, 'Customers')
Items = pd.read_excel(xlsx, 'Items')
Sales = pd.read_excel(xlsx, 'Sales')
Stores = pd.read_excel(xlsx, 'Stores')
Transactions = pd.read_excel(xlsx, 'Transactions')
ZipData = pd.read_excel(xlsx, 'ZipData')
Customer_Survey = pd.read_excel(xlsx, 'Customer_Survey')

In [3]:
# What is the shape of each dataframe?
print('Customers dataframe shape: ' + str(Customers.shape))
print('Items dataframe shape: ' + str(Items.shape))
print('Sales dataframe shape: ' + str(Sales.shape))
print('Stores dataframe shape: ' + str(Stores.shape))
print('Transactions dataframe shape: ' + str(Transactions.shape))
print('ZipData dataframe shape: ' + str(ZipData.shape))
print('Customer_Survey dataframe shape: ' + str(Customer_Survey.shape))

Customers dataframe shape: (3635, 8)
Items dataframe shape: (200, 3)
Sales dataframe shape: (32035, 4)
Stores dataframe shape: (12, 8)
Transactions dataframe shape: (576108, 4)
ZipData dataframe shape: (1692, 21)
Customer_Survey dataframe shape: (3635, 7)


### 1.1.2 Looking for repeated and/or missing values

In [4]:
#verifying there are no repeated values for: Customers, Items, Stores and Sales
print('Customers',len(Customers.Customer_ID.unique()), Customers.shape)
print('Customer survey', len(Customer_Survey.Customer_ID.unique()), Customer_Survey.shape)
print('Items',len(Items.Item_ID.unique()), Items.shape)
print('Stores', len(Stores.Store_ID.unique()), Stores.shape)
print('Sales', len(Sales.Sale_ID.unique()), Sales.shape)
print('ZipData', len(ZipData.zip_code.unique()), ZipData.shape)

Customers 3635 (3635, 8)
Customer survey 3635 (3635, 7)
Items 200 (200, 3)
Stores 12 (12, 8)
Sales 32035 (32035, 4)
ZipData 1692 (1692, 21)


##### All unique values match the number of rows in the dataframes

In [5]:
#checking for possible missing values in the data
print(Customers.info())
print(Items.info())
print(Sales.info())
print(Stores.info())
print(Transactions.info())
print(Customer_Survey.info())
print(ZipData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3635 entries, 0 to 3634
Data columns (total 8 columns):
Customer_ID              3635 non-null int64
First_Name               3635 non-null object
Last_Name                3635 non-null object
Customer_Address         3635 non-null object
Customer_City            3635 non-null object
Customer_State           3635 non-null object
Customer_Zip             3635 non-null int64
Customer_Phone_Number    3635 non-null object
dtypes: int64(2), object(6)
memory usage: 227.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
Item_ID           200 non-null int64
Item_Name         200 non-null object
Price_Per_Item    200 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32035 entries, 0 to 32034
Data columns (total 4 columns):
Sale_ID        32035 non-null int64
Customer_ID    32035 non-null int64
Store_ID       32035

##### There are null values in the ZipData dataframe. Since every row matches a specific Zip code, removing this rows might run the risk of eliminating a zip code that belongs to a specific customer. For this reason, it is best to replace missing values with the average value of every column.

In [6]:
ZipData.head()

,zip_code,City,State,num_households,median_income,percentage_white,percentage_black,percentage_american_indian,percentage_asian,percentage_pacific_islander,...,percentage_two_races,percentage_householder_age_15_to,percentage_householder_25_to_44,percentage_householder_45_to_64,percenage_householder_65_up,percentage_of_families_with_chil,percentage_of_families_with_no_c,percentage_married_couple_famili,percentage_female_householder,percentage_male_householder
0,1001,Agawam,MA,7103,60161,93.0,1.4,0.3,2.3,0.0,...,1.4,2.8,27.6,40.5,29.2,37.7,62.3,79.6,13.2,7.2
1,1002,Amherst,MA,9541,50540,81.1,5.3,0.5,11.9,0.0,...,0.9,22.9,25.2,29.9,22.0,41.7,58.3,77.5,17.4,5.1
2,1003,Amherst,MA,29,(X),58.6,0.0,41.4,0.0,0.0,...,0.0,17.2,41.4,41.4,0.0,100.0,0.0,100.0,0.0,0.0
3,1005,Barre,MA,1812,68786,99.1,0.0,0.0,0.9,0.0,...,0.0,1.2,21.8,46.5,30.6,45.0,55.0,78.0,12.6,9.3
4,1007,Belchertown,MA,5716,76881,96.6,1.0,0.0,1.5,0.0,...,0.2,2.3,25.2,50.3,22.2,50.0,50.0,80.2,14.1,5.7


In [7]:
#Check ZipData for missing values and replace with average income for the column
try:
    ZipData['median_income']= ZipData['median_income'].astype(float) #fixing the type of the median income column
except ValueError:
    pass #ignore incorrect and missing values

#convert all non-numerical values to NaN
data_columns = ZipData.columns[3:]
ZipData = (ZipData.drop(data_columns, axis=1)
        .join(ZipData[data_columns].apply(pd.to_numeric, errors='coerce'))) 
ZipData.head()

,zip_code,City,State,num_households,median_income,percentage_white,percentage_black,percentage_american_indian,percentage_asian,percentage_pacific_islander,...,percentage_two_races,percentage_householder_age_15_to,percentage_householder_25_to_44,percentage_householder_45_to_64,percenage_householder_65_up,percentage_of_families_with_chil,percentage_of_families_with_no_c,percentage_married_couple_famili,percentage_female_householder,percentage_male_householder
0,1001,Agawam,MA,7103,60161.0,93.0,1.4,0.3,2.3,0.0,...,1.4,2.8,27.6,40.5,29.2,37.7,62.3,79.6,13.2,7.2
1,1002,Amherst,MA,9541,50540.0,81.1,5.3,0.5,11.9,0.0,...,0.9,22.9,25.2,29.9,22.0,41.7,58.3,77.5,17.4,5.1
2,1003,Amherst,MA,29,NaN,58.6,0.0,41.4,0.0,0.0,...,0.0,17.2,41.4,41.4,0.0,100.0,0.0,100.0,0.0,0.0
3,1005,Barre,MA,1812,68786.0,99.1,0.0,0.0,0.9,0.0,...,0.0,1.2,21.8,46.5,30.6,45.0,55.0,78.0,12.6,9.3
4,1007,Belchertown,MA,5716,76881.0,96.6,1.0,0.0,1.5,0.0,...,0.2,2.3,25.2,50.3,22.2,50.0,50.0,80.2,14.1,5.7


In [8]:
print(ZipData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692 entries, 0 to 1691
Data columns (total 21 columns):
zip_code                            1692 non-null int64
City                                1692 non-null object
State                               1692 non-null object
num_households                      1692 non-null int64
median_income                       1605 non-null float64
percentage_white                    1659 non-null float64
percentage_black                    1659 non-null float64
percentage_american_indian          1659 non-null float64
percentage_asian                    1659 non-null float64
percentage_pacific_islander         1659 non-null float64
percentage_other_race               1659 non-null float64
percentage_two_races                1659 non-null float64
percentage_householder_age_15_to    1659 non-null float64
percentage_householder_25_to_44     1659 non-null float64
percentage_householder_45_to_64     1659 non-null float64
percenage_householder_65_up  

In [9]:
ZipData_averages  = {} #empty dictionary to fill with average value of every column in the ZipData DataFrame
#finding the average of every column and appending to averages list
for column in ZipData.columns[3:]: #only finding the average for columns with numerical values
    column_avg = round(np.mean(ZipData[column]),2)
    ZipData_averages[column] = column_avg

In [10]:
ZipData_averages

{'num_households': 4540.77,
 'median_income': 49060.07,
 'percentage_white': 81.55,
 'percentage_black': 12.62,
 'percentage_american_indian': 0.58,
 'percentage_asian': 2.0,
 'percentage_pacific_islander': 0.06,
 'percentage_other_race': 1.66,
 'percentage_two_races': 1.53,
 'percentage_householder_age_15_to': 2.88,
 'percentage_householder_25_to_44': 28.49,
 'percentage_householder_45_to_64': 41.88,
 'percenage_householder_65_up': 26.74,
 'percentage_of_families_with_chil': 38.75,
 'percentage_of_families_with_no_c': 61.25,
 'percentage_married_couple_famili': 75.69,
 'percentage_female_householder': 17.8,
 'percentage_male_householder': 6.51}

In [11]:
# Replace NaN values with column average:
ZipData.fillna(value=ZipData_averages, inplace = True)

In [12]:
ZipData.isna().sum() #check all NaN have been replaced with column average

zip_code                            0
City                                0
State                               0
num_households                      0
median_income                       0
percentage_white                    0
percentage_black                    0
percentage_american_indian          0
percentage_asian                    0
percentage_pacific_islander         0
percentage_other_race               0
percentage_two_races                0
percentage_householder_age_15_to    0
percentage_householder_25_to_44     0
percentage_householder_45_to_64     0
percenage_householder_65_up         0
percentage_of_families_with_chil    0
percentage_of_families_with_no_c    0
percentage_married_couple_famili    0
percentage_female_householder       0
percentage_male_householder         0
dtype: int64

In [13]:
ZipData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692 entries, 0 to 1691
Data columns (total 21 columns):
zip_code                            1692 non-null int64
City                                1692 non-null object
State                               1692 non-null object
num_households                      1692 non-null int64
median_income                       1692 non-null float64
percentage_white                    1692 non-null float64
percentage_black                    1692 non-null float64
percentage_american_indian          1692 non-null float64
percentage_asian                    1692 non-null float64
percentage_pacific_islander         1692 non-null float64
percentage_other_race               1692 non-null float64
percentage_two_races                1692 non-null float64
percentage_householder_age_15_to    1692 non-null float64
percentage_householder_25_to_44     1692 non-null float64
percentage_householder_45_to_64     1692 non-null float64
percenage_householder_65_up  

##### All null-values have been replace with column Averages

### 1.1.3 Fix the datatypes in each column of the dataframes

In [14]:
Customers['Customer_ID'] = Customers['Customer_ID'].astype(int) #customer ID should be integer for sorting
Customers['Customer_Zip']= Customers['Customer_Zip'].astype(str) # Customer Zip code should be categorical
Items['Item_ID']=Items['Item_ID'].astype(int) #Item ID should be integer for sorting
Items['Price_Per_Item'] = Items['Price_Per_Item'].astype(float) # Prices should be floats
Sales['Sale_ID'] = Sales['Sale_ID'].astype(int) #IDs should be integer for sorting
Sales['Customer_ID'] = Sales['Customer_ID'].astype(int) #IDs should be integer for sorting
Sales['Store_ID'] = Sales['Store_ID'].astype(int) #IDs should be integer for sorting
Stores['Store_ID'] = Stores['Store_ID'].astype(int) #IDs should be integer for sorting
Stores['Store_Zip'] = Stores['Store_Zip'].astype(int) #Zip codes should be integer for sorting
Transactions['Sale_ID'] = Transactions['Sale_ID'].astype(int) #IDs should be integer for sortingl
Transactions['Item_ID'] = Transactions['Item_ID'].astype(int) #IDs should be integer for sorting
Transactions['Amount_Purchased'] = Transactions['Amount_Purchased'].astype(float) #some groceries can have decimal quantities
Customer_Survey['Customer_ID'] = Customer_Survey['Customer_ID'].astype(int) #IDs should be integer for sortingl
Customer_Survey['Cust_Income'] = Customer_Survey['Cust_Income'].astype(float) #income can be a decimal value
ZipData['zip_code'] = ZipData['zip_code'].astype(int) #Zip codes should be integer for sorting

## 1.2 American Census Data

### 1.2.1 Reading the data

In [15]:
# Read in US Census American Community Survey (ACS)
ACS = pd.read_csv('American Community Survey\ACS_17_5YR_S1501\ACS_17_5YR_S1501_with_ann.csv', low_memory = False)

In [16]:
ACS.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC03_EST_VC02,HC03_MOE_VC02,HC04_EST_VC02,...,HC02_EST_VC85,HC02_MOE_VC85,HC03_EST_VC85,HC03_MOE_VC85,HC04_EST_VC85,HC04_MOE_VC85,HC05_EST_VC85,HC05_MOE_VC85,HC06_EST_VC85,HC06_MOE_VC85
0,Id,Id2,Geography,Total; Estimate; Population 18 to 24 years,Total; Margin of Error; Population 18 to 24 years,Percent; Estimate; Population 18 to 24 years,Percent; Margin of Error; Population 18 to 24 ...,Male; Estimate; Population 18 to 24 years,Male; Margin of Error; Population 18 to 24 years,Percent Male; Estimate; Population 18 to 24 years,...,Percent; Estimate; MEDIAN EARNINGS IN THE PAST...,Percent; Margin of Error; MEDIAN EARNINGS IN T...,Male; Estimate; MEDIAN EARNINGS IN THE PAST 12...,Male; Margin of Error; MEDIAN EARNINGS IN THE ...,Percent Male; Estimate; MEDIAN EARNINGS IN THE...,Percent Male; Margin of Error; MEDIAN EARNINGS...,Female; Estimate; MEDIAN EARNINGS IN THE PAST ...,Female; Margin of Error; MEDIAN EARNINGS IN TH...,Percent Female; Estimate; MEDIAN EARNINGS IN T...,Percent Female; Margin of Error; MEDIAN EARNIN...
1,8600000US00601,00601,ZCTA5 00601,1693,88,(X),(X),854,82,(X),...,(X),(X),36417,10986,(X),(X),33681,15268,(X),(X)
2,8600000US00602,00602,ZCTA5 00602,3864,34,(X),(X),1953,30,(X),...,(X),(X),35547,12476,(X),(X),26572,2397,(X),(X)
3,8600000US00603,00603,ZCTA5 00603,4620,234,(X),(X),2405,161,(X),...,(X),(X),46806,11648,(X),(X),31698,3513,(X),(X)
4,8600000US00606,00606,ZCTA5 00606,632,52,(X),(X),326,20,(X),...,(X),(X),-,**,(X),(X),-,**,(X),(X)


In [17]:
 # Set the column names to be the contents of the first row (zero index).
ACS.index=ACS['GEO.id2'] # Set the row names (index names) to be the contents of the column with label 'Id2'.
ACS=ACS.iloc[1:] # Eliminate the first row.
ACS.drop('GEO.id2', axis=1, inplace=True) # Eliminate the first column.
ACS['Zipcode'] = pd.to_numeric(ACS['GEO.id'].str[-5:]) #adding column with numeric zip code ID
ACS['ZipCodeString'] = ACS['GEO.display-label'].str.split(n = 1).str[1] #adding column with Zipcode as string

In [18]:
ACS.head()

,GEO.id,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC03_EST_VC02,HC03_MOE_VC02,HC04_EST_VC02,HC04_MOE_VC02,...,HC03_EST_VC85,HC03_MOE_VC85,HC04_EST_VC85,HC04_MOE_VC85,HC05_EST_VC85,HC05_MOE_VC85,HC06_EST_VC85,HC06_MOE_VC85,Zipcode,ZipCodeString
GEO.id2,,,,,,,,,,,,,,,,,,,,,
00601,8600000US00601,ZCTA5 00601,1693,88,(X),(X),854,82,(X),(X),...,36417,10986,(X),(X),33681,15268,(X),(X),601,00601
00602,8600000US00602,ZCTA5 00602,3864,34,(X),(X),1953,30,(X),(X),...,35547,12476,(X),(X),26572,2397,(X),(X),602,00602
00603,8600000US00603,ZCTA5 00603,4620,234,(X),(X),2405,161,(X),(X),...,46806,11648,(X),(X),31698,3513,(X),(X),603,00603
00606,8600000US00606,ZCTA5 00606,632,52,(X),(X),326,20,(X),(X),...,-,**,(X),(X),-,**,(X),(X),606,00606
00610,8600000US00610,ZCTA5 00610,2703,30,(X),(X),1412,2,(X),(X),...,65143,55270,(X),(X),27083,10149,(X),(X),610,00610


In [19]:
print('ACS dataframe shape: '+str(ACS.shape))

ACS dataframe shape: (33120, 772)


### 1.2.2 Keeping only the required columns for the linear regression model

As indicated above, from the ACS dataset we only need the columns containing the population poverty and percentage of people over 25 years for every zipcode. We also only need the columns that contain the aggregated version of these two features, and disregards the columns separating between male and female, and education level. 

**The % of the population over 25 years old can be calculated as:**
\begin{equation}
fraction​population​over​25​yrs = \frac{​​total​population​over​25​years​​}{total​population​over​25​years+total​population​from​18​to​24 years}
\end{equation}  
For this we need columns:
'HC01_EST_VC02 : Total; Estimate; Population 18 to 24 years'
'HC01_EST_VC08: Total; Estimate; Population 25 years and over'

__________________________________________________
**The aggregated poverty rate of the entire population for a given zipcode can be calculated as the mean value from the following columns in the ACS dataset:** 
* 'HC02_EST_VC74: Percentage; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Less than high school graduate'
* 'HC02_EST_VC75: Percentage; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - High school graduate (includes equivalency)'
* 'HC02_EST_VC76: Percentage; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Some college or associate's degree'
* 'HC02_EST_VC77': Percentage; Estimate; POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL - Bachelor's degree or higher'


In [20]:
# creating the new dataframe which contains only the desired columns
ACS_2 = pd.DataFrame()
ACS_2['Zip_code'] = ACS['Zipcode']
ACS_2['Zip_code_string'] = ACS['ZipCodeString']
ACS_2['HC01_EST_VC02'] = ACS['HC01_EST_VC02']
ACS_2['HC01_EST_VC08'] = ACS['HC01_EST_VC08']
ACS_2['HC02_EST_VC74'] = ACS['HC02_EST_VC74']
ACS_2['HC02_EST_VC75'] = ACS['HC02_EST_VC75']
ACS_2['HC02_EST_VC76'] = ACS['HC02_EST_VC76']
ACS_2['HC02_EST_VC77'] = ACS['HC02_EST_VC77']

In [21]:
print(len(ACS_2.Zip_code.unique()), ACS_2.shape) #checking for duplicate zip codes in the data

33120 (33120, 8)


In [22]:
ACS_2.tail()

,Zip_code,Zip_code_string,HC01_EST_VC02,HC01_EST_VC08,HC02_EST_VC74,HC02_EST_VC75,HC02_EST_VC76,HC02_EST_VC77
GEO.id2,,,,,,,,
99923,99923,99923,0,0,-,-,-,-
99925,99925,99925,79,647,33.7,17.1,18.7,6.8
99926,99926,99926,147,1029,13.1,14.1,5.0,2.6
99927,99927,99927,0,59,-,100.0,100.0,47.4
99929,99929,99929,137,1837,8.0,11.8,10.2,0.0


##### The head of the dataframe we see some values marked with '-', these will be transformed to NaN

In [23]:
#convert all non-numerical values to NaN
data_columns = ACS_2.columns[3:]
ACS_2 = (ACS_2.drop(data_columns, axis=1)
        .join(ACS_2[data_columns].apply(pd.to_numeric, errors='coerce'))) 
ACS_2.tail()

,Zip_code,Zip_code_string,HC01_EST_VC02,HC01_EST_VC08,HC02_EST_VC74,HC02_EST_VC75,HC02_EST_VC76,HC02_EST_VC77
GEO.id2,,,,,,,,
99923,99923,99923,0,0,NaN,NaN,NaN,NaN
99925,99925,99925,79,647,33.7,17.1,18.7,6.8
99926,99926,99926,147,1029,13.1,14.1,5.0,2.6
99927,99927,99927,0,59,NaN,100.0,100.0,47.4
99929,99929,99929,137,1837,8.0,11.8,10.2,0.0


In [24]:
#check for NaN values
ACS_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33120 entries, 00601 to 99929
Data columns (total 8 columns):
Zip_code           33120 non-null int64
Zip_code_string    33120 non-null object
HC01_EST_VC02      33120 non-null object
HC01_EST_VC08      33120 non-null int64
HC02_EST_VC74      31185 non-null float64
HC02_EST_VC75      32201 non-null float64
HC02_EST_VC76      32133 non-null float64
HC02_EST_VC77      31587 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 3.5+ MB


In [25]:
#Calculate averages of the poverty rate columns
ACS_VC02_avg = round(np.mean(ACS_2['HC01_EST_VC02']))
ACS_VC08_avg = round(np.mean(ACS_2['HC01_EST_VC08']))
ACS_VC74_avg = round(np.mean(ACS_2['HC02_EST_VC74']),2)
ACS_VC75_avg = round(np.mean(ACS_2['HC02_EST_VC75']),2)
ACS_VC76_avg = round(np.mean(ACS_2['HC02_EST_VC76']),2)
ACS_VC77_avg = round(np.mean(ACS_2['HC02_EST_VC77']),2)

In [26]:
#Replace NaN values with the average of the column
ACS_2['HC01_EST_VC02'].fillna(ACS_VC02_avg, inplace = True)
ACS_2['HC01_EST_VC08'].fillna(ACS_VC08_avg, inplace = True)
ACS_2['HC02_EST_VC74'].fillna(ACS_VC74_avg, inplace = True)
ACS_2['HC02_EST_VC75'].fillna(ACS_VC75_avg, inplace = True)
ACS_2['HC02_EST_VC76'].fillna(ACS_VC76_avg, inplace = True)
ACS_2['HC02_EST_VC77'].fillna(ACS_VC77_avg, inplace = True)

In [27]:
ACS_2.tail()

,Zip_code,Zip_code_string,HC01_EST_VC02,HC01_EST_VC08,HC02_EST_VC74,HC02_EST_VC75,HC02_EST_VC76,HC02_EST_VC77
GEO.id2,,,,,,,,
99923,99923,99923,0,0,23.72,13.75,10.7,5.23
99925,99925,99925,79,647,33.70,17.10,18.7,6.80
99926,99926,99926,147,1029,13.10,14.10,5.0,2.60
99927,99927,99927,0,59,23.72,100.00,100.0,47.40
99929,99929,99929,137,1837,8.00,11.80,10.2,0.00


##### All missing values have been replaced with the column average

In [28]:
#Change dtypes to int and floats
ACS_2['HC01_EST_VC02'] = ACS_2['HC01_EST_VC02'].astype('int64') # population should be int
ACS_2['HC01_EST_VC08'] = ACS_2['HC01_EST_VC08'].astype('int64') # population should be int
ACS_2['HC02_EST_VC74'] = ACS_2['HC02_EST_VC74'].astype(float) # poverty rate should be a float
ACS_2['HC02_EST_VC75'] = ACS_2['HC02_EST_VC75'].astype(float) # poverty rate should be a float
ACS_2['HC02_EST_VC76'] = ACS_2['HC02_EST_VC76'].astype(float) # poverty rate should be a float
ACS_2['HC02_EST_VC77'] = ACS_2['HC02_EST_VC77'].astype(float) # poverty rate should be a float

In [29]:
ACS_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33120 entries, 00601 to 99929
Data columns (total 8 columns):
Zip_code           33120 non-null int64
Zip_code_string    33120 non-null object
HC01_EST_VC02      33120 non-null int64
HC01_EST_VC08      33120 non-null int64
HC02_EST_VC74      33120 non-null float64
HC02_EST_VC75      33120 non-null float64
HC02_EST_VC76      33120 non-null float64
HC02_EST_VC77      33120 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 3.5+ MB


#### 1.2.2.1 Calculating the percentage of the population over 25 years old:

\begin{equation}
fraction​population​over​25​yrs = \frac{​​total​population​over​25​years​​}{total​population​over​25​years+total​population​from​18​to​24 years}
\end{equation}  
For this we need columns:
'HC01_EST_VC02 : Total; Estimate; Population 18 to 24 years'
'HC01_EST_VC08: Total; Estimate; Population 25 years and over'

In [30]:
# function to calculate the population fraction of people over 25 yrs old
def fraction(df, columns):
    '''
    function to calculate the fraction between two columns in a dataframe
    arguments: 
    df: a df containing the two columns to calculate the fraction from
    columns: name of the columns to use in the calculation. column[0] must be the aimed fraction value
    return: the fraction calculated as a/(a+b)
    '''
    fraction = 0
    if (df[columns[0]]+df[columns[1]]) != 0:
        try:
            fraction = round(df[columns[0]]/(df[columns[0]]+df[columns[1]]),2)
        except TypeError:
            pass
    return fraction

In [31]:
# Calculate for each zip code the percentage of the population over the age of 25
columns = ['HC01_EST_VC08','HC01_EST_VC02']
ACS_2['%Population over 25 years'] = ACS_2.apply(lambda x: fraction(x,columns), axis =1)

In [32]:
ACS_2.tail()

,Zip_code,Zip_code_string,HC01_EST_VC02,HC01_EST_VC08,HC02_EST_VC74,HC02_EST_VC75,HC02_EST_VC76,HC02_EST_VC77,%Population over 25 years
GEO.id2,,,,,,,,,
99923,99923,99923,0,0,23.72,13.75,10.7,5.23,0.00
99925,99925,99925,79,647,33.70,17.10,18.7,6.80,0.89
99926,99926,99926,147,1029,13.10,14.10,5.0,2.60,0.88
99927,99927,99927,0,59,23.72,100.00,100.0,47.40,1.00
99929,99929,99929,137,1837,8.00,11.80,10.2,0.00,0.93


In [33]:
#How many zip codes have 0% population over 25 years and how many have 100%?
ACS_pop_zero = ACS_2['%Population over 25 years'].isin([0]).sum()
per_pop_zero = round((ACS_pop_zero/len(ACS_2.Zip_code.unique()))*100,2)
ACS_pop_one = ACS_2['%Population over 25 years'].isin([1]).sum()
per_pop_one = round(ACS_pop_one/len(ACS_2.Zip_code.unique())*100,2)
print("Number of zipcodes where the population over age 25 was 0: " + str(ACS_pop_zero)+',', str(per_pop_zero)+'% of the entire dataset')
print("Number of zipcodes where the population over age 25 was 100%: " + str(ACS_pop_one)+',', str(per_pop_one)+'% of the entire dataset')

Number of zipcodes where the population over age 25 was 0: 380, 1.15% of the entire dataset
Number of zipcodes where the population over age 25 was 100%: 1965, 5.93% of the entire dataset


#### 1.2.2.2 Calculating the aggregated poverty rate for each zip code population

In [34]:
#Average the poverty rate of all education levels for each zip code
ACS_2['%Poverty_Average'] = ACS_2.iloc[:,4:8].mean(axis=1)

In [35]:
ACS_2.tail()

,Zip_code,Zip_code_string,HC01_EST_VC02,HC01_EST_VC08,HC02_EST_VC74,HC02_EST_VC75,HC02_EST_VC76,HC02_EST_VC77,%Population over 25 years,%Poverty_Average
GEO.id2,,,,,,,,,,
99923,99923,99923,0,0,23.72,13.75,10.7,5.23,0.00,13.350
99925,99925,99925,79,647,33.70,17.10,18.7,6.80,0.89,19.075
99926,99926,99926,147,1029,13.10,14.10,5.0,2.60,0.88,8.700
99927,99927,99927,0,59,23.72,100.00,100.0,47.40,1.00,67.780
99929,99929,99929,137,1837,8.00,11.80,10.2,0.00,0.93,7.500


In [36]:
# ACS_final data frame with percent of population over 25 and poverty average per zip code
ACS_final = pd.DataFrame()
ACS_final['Zip_code'] = ACS_2['Zip_code']
ACS_final['Zip_code_string'] = ACS_2['Zip_code_string']
ACS_final['%Population over 25 years'] = ACS_2['%Population over 25 years']
ACS_final['%Poverty_Average'] = ACS_2['%Poverty_Average']
ACS_final.head()

,Zip_code,Zip_code_string,%Population over 25 years,%Poverty_Average
GEO.id2,,,,
00601,601,00601,0.88,55.225
00602,602,00602,0.88,45.575
00603,603,00603,0.88,44.800
00606,606,00606,0.87,48.900
00610,610,00610,0.88,36.700


## 2. Merge all tables together in a single sales DataFrame following the schema


In [37]:
# Merge Customers and Customers_Survery dataframes together
Customers_Customers_Survey = pd.merge(Customers, Customer_Survey, on = 'Customer_ID', how = 'left')

# Merge Items and Transactions dataframe together 
Items_Transactions = pd.merge(Transactions, Items, on = 'Item_ID', how = 'left')

# Merge ACS_final data frame and ZipData dataframe together
ZipData['zip_code'] = ZipData['zip_code'].astype('int64')
ZipData_ACS_final = pd.merge(ZipData, ACS_final, left_on = 'zip_code', right_on = "Zip_code", how = 'left')
ZipData_ACS_final = ZipData_ACS_final.drop(columns=['Zip_code','Zip_code_string'], axis = 'columns')

#Merge Stores and ZipData_ACS_final
Stores['Store_Zip'] = Stores['Store_Zip'].astype('int64')
Stores_ZipData_ACS_final = pd.merge(Stores, ZipData_ACS_final, left_on = 'Store_Zip', right_on = "zip_code", how = 'left')

# Merge Sales and Stores_ZipData_ACS_final dataframes together
Sales_Stores_ZipData_ACS_final = pd.merge(Sales, Stores_ZipData_ACS_final, on = 'Store_ID', how = 'left')

# Merge Sales_Stores_ZipData_ACS_final and Customers_Customers_Survey dataframes together
Sales_Stores_ZipData_ACS_final_Customers_Customers_Survey = pd.merge(Sales_Stores_ZipData_ACS_final, Customers_Customers_Survey, on = 'Customer_ID', how = 'left')

# Merge Sales_Stores_ZipData_final_Customers_Customers_Survey and Items_Transactions into final dataframe df
df = pd.merge(Sales_Stores_ZipData_ACS_final_Customers_Customers_Survey, Items_Transactions, on = 'Sale_ID', how = 'left')

In [38]:
#final df columns and shapes
print(df.columns)
print(df.shape)
df.head()

Index(['Sale_ID', 'Customer_ID', 'Store_ID', 'Sale_Week', 'Store_Name',
       'Store_Size', 'Store_Address', 'Store_City', 'Store_State', 'Store_Zip',
       'Store_Phone_Number', 'zip_code', 'City', 'State', 'num_households',
       'median_income', 'percentage_white', 'percentage_black',
       'percentage_american_indian', 'percentage_asian',
       'percentage_pacific_islander', 'percentage_other_race',
       'percentage_two_races', 'percentage_householder_age_15_to',
       'percentage_householder_25_to_44', 'percentage_householder_45_to_64',
       'percenage_householder_65_up', 'percentage_of_families_with_chil',
       'percentage_of_families_with_no_c', 'percentage_married_couple_famili',
       'percentage_female_householder', 'percentage_male_householder',
       '%Population over 25 years', '%Poverty_Average', 'First_Name',
       'Last_Name', 'Customer_Address', 'Customer_City', 'Customer_State',
       'Customer_Zip', 'Customer_Phone_Number', 'Cust_Sex', 'Cust_Income',


,Sale_ID,Customer_ID,Store_ID,Sale_Week,Store_Name,Store_Size,Store_Address,Store_City,Store_State,Store_Zip,...,Cust_Income,Cust_Race,Cust_Age,Cust_Children,Cust_Rel_Status,Item_ID,Amount_Purchased,Item_Discount,Item_Name,Price_Per_Item
0,1,2719870,11,1,Bill's Barter,Medium,887 Navigators Street,Malden,MA,2148,...,20000.0,White,45-64,Has_no_Child(ren),Married,146932,2.0,0.0,Raspberries,4.0
1,1,2719870,11,1,Bill's Barter,Medium,887 Navigators Street,Malden,MA,2148,...,20000.0,White,45-64,Has_no_Child(ren),Married,428188,2.0,0.0,Mustard,2.0
2,1,2719870,11,1,Bill's Barter,Medium,887 Navigators Street,Malden,MA,2148,...,20000.0,White,45-64,Has_no_Child(ren),Married,496931,1.0,0.0,Clam Chowder,3.0
3,1,2719870,11,1,Bill's Barter,Medium,887 Navigators Street,Malden,MA,2148,...,20000.0,White,45-64,Has_no_Child(ren),Married,564177,3.0,0.0,Cuman,3.0
4,1,2719870,11,1,Bill's Barter,Medium,887 Navigators Street,Malden,MA,2148,...,20000.0,White,45-64,Has_no_Child(ren),Married,579017,3.0,0.0,Nutmeg,3.0


## 3. Group all sales by Sale_ID

### 3.1 Calculate the total amount of a sale

In [39]:
# creating a column with Total sale amount per Sales ID
df['Purchase_Total']= df.apply(lambda x: (x['Amount_Purchased']*x['Price_Per_Item'])*(1-x['Item_Discount']),axis=1)

### 3.2 Group sales by Sale_ID

In [40]:
grouped_sales = df['Purchase_Total'].groupby([df['Sale_ID'], df['Customer_ID'], df['Store_ID'], df['Cust_Income'], 
                                              df['Cust_Race'], df['Cust_Children'], df['Store_Size'],
                                              df['%Poverty_Average'], df['%Population over 25 years']]).sum().reset_index(name = 'Sale amount').sort_values(['Sale_ID'], ascending = True)

In [41]:
grouped_sales

,Sale_ID,Customer_ID,Store_ID,Cust_Income,Cust_Race,Cust_Children,Store_Size,%Poverty_Average,%Population over 25 years,Sale amount
0,1,2719870,11,20000.0,White,Has_no_Child(ren),Medium,14.575,0.88,64.7
1,2,5678891,1,45900.0,Black,Has_Child(ren),Large,13.350,0.00,67.7
2,3,6436421,3,55600.0,White,Has_Child(ren),Small,5.575,0.91,121.0
3,4,2829790,10,90100.0,Asian,Has_no_Child(ren),Large,18.850,0.88,139.8
4,5,3559824,6,47100.0,Other,Has_Child(ren),Small,17.975,0.88,57.1
...,...,...,...,...,...,...,...,...,...,...
32030,32031,4683119,8,20000.0,White,Has_Child(ren),Small,16.775,0.84,27.4
32031,32032,3597103,9,83400.0,Black,Has_no_Child(ren),Large,41.375,0.87,123.1
32032,32033,2676201,11,95800.0,White,Has_no_Child(ren),Medium,14.575,0.88,181.3
32033,32034,3558958,8,73000.0,White,Has_no_Child(ren),Small,16.775,0.84,180.2


## 4. Create "X" and "y" DataFrames for the ML model

In [42]:
y = grouped_sales['Sale amount']
y

0         64.7
1         67.7
2        121.0
3        139.8
4         57.1
         ...  
32030     27.4
32031    123.1
32032    181.3
32033    180.2
32034    158.4
Name: Sale amount, Length: 32035, dtype: float64

In [43]:
X = grouped_sales.drop(['Sale_ID','Customer_ID','Store_ID','Sale amount'], axis =1)
X

,Cust_Income,Cust_Race,Cust_Children,Store_Size,%Poverty_Average,%Population over 25 years
0,20000.0,White,Has_no_Child(ren),Medium,14.575,0.88
1,45900.0,Black,Has_Child(ren),Large,13.350,0.00
2,55600.0,White,Has_Child(ren),Small,5.575,0.91
3,90100.0,Asian,Has_no_Child(ren),Large,18.850,0.88
4,47100.0,Other,Has_Child(ren),Small,17.975,0.88
...,...,...,...,...,...,...
32030,20000.0,White,Has_Child(ren),Small,16.775,0.84
32031,83400.0,Black,Has_no_Child(ren),Large,41.375,0.87
32032,95800.0,White,Has_no_Child(ren),Medium,14.575,0.88
32033,73000.0,White,Has_no_Child(ren),Small,16.775,0.84


## 5. Apply one-hot encoding to categorical variables

In [44]:
X = pd.get_dummies(data=X, columns =['Cust_Race','Cust_Children','Store_Size'])
X

,Cust_Income,%Poverty_Average,%Population over 25 years,Cust_Race_American_Indian,Cust_Race_Asian,Cust_Race_Black,Cust_Race_Other,Cust_Race_Two_or_More,Cust_Race_White,Cust_Children_Has_Child(ren),Cust_Children_Has_no_Child(ren),Store_Size_Large,Store_Size_Medium,Store_Size_Small
0,20000.0,14.575,0.88,0,0,0,0,0,1,0,1,0,1,0
1,45900.0,13.350,0.00,0,0,1,0,0,0,1,0,1,0,0
2,55600.0,5.575,0.91,0,0,0,0,0,1,1,0,0,0,1
3,90100.0,18.850,0.88,0,1,0,0,0,0,0,1,1,0,0
4,47100.0,17.975,0.88,0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32030,20000.0,16.775,0.84,0,0,0,0,0,1,1,0,0,0,1
32031,83400.0,41.375,0.87,0,0,1,0,0,0,0,1,1,0,0
32032,95800.0,14.575,0.88,0,0,0,0,0,1,0,1,0,1,0
32033,73000.0,16.775,0.84,0,0,0,0,0,1,0,1,0,0,1


## 6. Standardize numerical independent variables

Standardarization is done only in columns containing numerical values

In [45]:
from sklearn.preprocessing import StandardScaler
#separate X into columns with numerical and categorical values
X_num = X.iloc[:,:3]
X_cat = X.iloc[:,3:]


In [46]:
# Standardize the numerical portion of X
X_standard_num = StandardScaler().fit_transform(X_num)
X_df_standard_num = pd.DataFrame(data=X_standard[:,:], columns=X_num.columns)
X_df_standard_num

NameError: name 'X_standard' is not defined

In [ ]:
#concatenate with the categorical portion of x
X_standard = pd.concat([X_df_standard_num,X_cat], axis =1)

In [ ]:
X_standard